In [67]:
from bs4 import BeautifulSoup
import re
import time
import requests
import csv

def scrape(url):
    
    output=open('reviews.csv', 'w', encoding='utf8')
    
    writer=csv.writer(output,lineterminator='\n')
    
    for i in range(5):
        response=requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36', })
        if response:
            break
        else:
            print('fail',i)
            time.sleep(5)
            
    if not response: 
        return None
    
    html=response.text
    soup=BeautifulSoup(html,'html')
    
    reviews=soup.findAll('li',{'class':re.compile('audience-reviews')})
    for review in reviews:
        
        critic,rate,date,text='NA','NA','NA','NA'
        criticChunk=review.find('span', {'class':'audience-reviews__name'})
        if criticChunk:
            critic=criticChunk.text.strip()
            
        rateChunk=review.find('span',{'class':'audience-reviews__score'})
        if rateChunk:
            stars_filled=rateChunk.findAll('span',{'class':('star-display__filled')})
            stars_half=rateChunk.findAll('span',{'class':('star-display__half')})   
            rate=len(stars_filled)+(len(stars_half)/2)
            
        dateChunk=review.find('span',{'class':'audience-reviews__duration'})
        if dateChunk:
            date=dateChunk.text.strip()
        
        textChunk=review.find('p',{'class':'audience-reviews__review js-review-text clamp clamp-8 js-clamp'})
        if textChunk:
            text=textChunk.text.strip()
        
        writer.writerow([critic,rate,date,text])
    
    output.close()
        

In [68]:
url='https://www.rottentomatoes.com/m/avengers_endgame/reviews?type=verified_audience'
scrape(url)